In [1]:
from datasets import load_dataset, Dataset, DatasetDict

/Users/dooterior/Desktop/projects/finetuning/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification, 
    DataCollatorWithPadding, 
    TrainingArguments, 
    Trainer)

In [3]:
import evaluate
import torch
import numpy as np

In [4]:
from peft import get_peft_model, LoraConfig

In [5]:
# load dataset
dataset = load_dataset('shawhin/imdb-truncated')
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 1000
    })
})

In [6]:
# display % of training data with label=1
np.array(dataset['train']['label']).sum()/len(dataset['train']['label'])

np.float64(0.5)

In [7]:
model_checkpoint = 'distilbert/distilbert-base-uncased'

# define label maps
id2label = {0: "Negative", 1: "Positive"}
label2id = {"Negative":0, "Positive":1}

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels =2, id2label=id2label, label2id=label2id)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


#### Preprocess the model/dataset

In [15]:
# initialize the tokenizer for the pretrained model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, add_prefix_space = True)

In [16]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

In [17]:
def tokenize(examples):
    text = examples['text']

    tokenizer.truncation_side = "left"
    tokenized_input = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=512
    )
    return tokenized_input

In [18]:
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset

Map: 100%|██████████| 1000/1000 [00:00<00:00, 5026.97 examples/s]


DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['label', 'text', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [19]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

#### Evaluation

In [20]:
# import accuracy evaluation metric
accuracy = evaluate.load("accuracy")

In [21]:
# define an evaluation function to pass into trainer later
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=1)

    return {"accuracy": accuracy.compute(predictions=predictions, references=labels)}

In [24]:
# define list of examples
text_list = ["It was good.", 
             "Really hated that ugghh", 
             "Not a fan, don't recommed.", 
             "Better than the first one.", 
             "This is not worth watching even once.", 
             "This one is a pass."]

print("Untrained model predictions:")
print("----------------------------")
for text in text_list:
    # tokenize text
    inputs = tokenizer.encode(text, return_tensors="pt")
    # compute logits
    logits = model(inputs).logits
    # convert logits to label
    predictions = torch.argmax(logits)

    print(text + " - " + id2label[predictions.tolist()])

Untrained model predictions:
----------------------------
It was good. - Positive
Really hated that ugghh - Positive
Not a fan, don't recommed. - Positive
Better than the first one. - Positive
This is not worth watching even once. - Positive
This one is a pass. - Positive


In [29]:
peft_config = LoraConfig(
    task_type = 'SEQ_CLS',
    r = 4,
    lora_alpha = 32,
    lora_dropout = 0.01,
    target_modules = ['q_lin']
   )
peft_config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type='SEQ_CLS', inference_mode=False, r=4, target_modules={'q_lin'}, lora_alpha=32, lora_dropout=0.01, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [30]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 628,994 || all params: 67,584,004 || trainable%: 0.9307


In [35]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 5

In [36]:
# define training arguments
training_args = TrainingArguments(
    output_dir= model_checkpoint + "-lora-text-classification",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [37]:
# creater trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

/var/folders/xy/b785rtgn795129srfj5kb0mc0000gn/T/ipykernel_2433/1491587621.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [38]:
# train model
trainer.train()



























































































































































































































































                                                 
                                               

  1%|▏         | 34/2500 [25:09<27:48,  1.48it/s]



{'eval_loss': 0.6318349242210388, 'eval_accuracy': {'accuracy': 0.849}, 'eval_runtime': 592.7249, 'eval_samples_per_second': 1.687, 'eval_steps_per_second': 0.422, 'epoch': 1.0}


                                                 
  1%|▏         | 34/2500 [43:37<27:48,  1.48it/s]   

{'loss': 0.4152, 'grad_norm': 4.117372989654541, 'learning_rate': 0.0006, 'epoch': 2.0}




























































































































































































































































                                                 
                                                 

  1%|▏         | 34/2500 [56:00<27:48,  1.48it/s]



...



























































































































































































































































                                                   
                                                  

  1%|▏         | 34/2500 [2:42:08<27:48,  1.48it/s]



{'eval_loss': 0.6998155117034912, 'eval_accuracy': {'accuracy': 0.898}, 'eval_runtime': 1081.4565, 'eval_samples_per_second': 0.925, 'eval_steps_per_second': 0.231, 'epoch': 5.0}


                                                   
100%|██████████| 1250/1250 [2:41:23<00:00,  7.75s/it]

{'train_runtime': 9683.0179, 'train_samples_per_second': 0.516, 'train_steps_per_second': 0.129, 'train_loss': 0.2241533317565918, 'epoch': 5.0}


TrainOutput(global_step=1250, training_loss=0.2241533317565918, metrics={'train_runtime': 9683.0179, 'train_samples_per_second': 0.516, 'train_steps_per_second': 0.129, 'total_flos': 556790525519424.0, 'train_loss': 0.2241533317565918, 'epoch': 5.0})

In [39]:
model.to('mps') # moving to mps for Mac (can alternatively do 'cpu')

print("Trained model predictions:")
print("--------------------------")
for text in text_list:
    inputs = tokenizer.encode(text, return_tensors="pt").to("mps") # moving to mps for Mac (can alternatively do 'cpu')

    logits = model(inputs).logits
    predictions = torch.max(logits,1).indices

    print(text + " - " + id2label[predictions.tolist()[0]])

Trained model predictions:
--------------------------
It was good. - Positive
Really hated that ugghh - Negative
Not a fan, don't recommed. - Negative
Better than the first one. - Positive
This is not worth watching even once. - Negative
This one is a pass. - Negative
